<a href="https://colab.research.google.com/github/cathay/cnn/blob/master/driver_action_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import random
import tensorflow as tf
from shutil import copyfile
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

#Import Google drive functions
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
import os, shutil
folder = '/content/drive/My Drive'
for i in range(100000):
  for filename in os.scandir(folder):
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              print(file_path)
              os.remove(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))

/content/drive/My Drive/img_83889.jpg
/content/drive/My Drive/._img_83889.jpg
/content/drive/My Drive/img_85220.jpg
/content/drive/My Drive/._img_85220.jpg
/content/drive/My Drive/img_81046.jpg
/content/drive/My Drive/._img_81046.jpg
/content/drive/My Drive/img_86729.jpg
/content/drive/My Drive/._img_86729.jpg
/content/drive/My Drive/img_87437.jpg
/content/drive/My Drive/._img_87437.jpg
/content/drive/My Drive/img_88704.jpg
/content/drive/My Drive/._img_88704.jpg
/content/drive/My Drive/img_80358.jpg
/content/drive/My Drive/._img_80358.jpg
/content/drive/My Drive/img_83679.jpg
/content/drive/My Drive/._img_83679.jpg
/content/drive/My Drive/img_84116.jpg
/content/drive/My Drive/._img_84116.jpg
/content/drive/My Drive/img_85208.jpg
/content/drive/My Drive/._img_85208.jpg
/content/drive/My Drive/img_82567.jpg
/content/drive/My Drive/._img_82567.jpg
/content/drive/My Drive/img_89432.jpg
/content/drive/My Drive/._img_89432.jpg
/content/drive/My Drive/img_86701.jpg
/content/drive/My Drive/._

In [0]:
#Authorize Google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#download train and test images
download = drive.CreateFile({'id': '1UPt8uX_wHzSplanU7zhGVDjsYpVMNikz'})
download.GetContentFile('state-farm-distracted-driver-detection.zip')

#Extract files
zip_ref = zipfile.ZipFile('state-farm-distracted-driver-detection.zip', 'r')
zip_ref.extractall('tmp')
zip_ref.close()

In [13]:
from keras.utils import to_categorical

def get_im_cv2(path, w, h):
  img = cv2.imread(path)
  resized = cv2.resize(img, (w,h), cv2.INTER_LINEAR) 
  return resized

TRAINING_DIR = '/content/tmp/state-farm-distracted-driver-detection/imgs/train'
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

files = []
categories = []
paths = []

for i in range(10):
  for file in os.listdir("%s/c%s" %(TRAINING_DIR, i)):
    files.append(file)
    categories.append(i)
    paths.append("%s/c%s/%s" %(TRAINING_DIR, i, file))

df = pd.DataFrame({'file': files,
                   'category': categories,
                   'path': paths
                  })

df['Y'] = df['category'].map(lambda x: to_categorical(x, 10, dtype='uint8'))
df['X'] = df['path'].map(lambda path: get_im_cv2(path, IMAGE_WIDTH, IMAGE_HEIGHT))
df.sample(5, axis=None)

,file,category,path,Y,X
16786,img_62010.jpg,7,/content/tmp/state-farm-distracted-driver-dete...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[[[41, 52, 44], [41, 52, 44], [41, 52, 44], [4..."
13166,img_37319.jpg,5,/content/tmp/state-farm-distracted-driver-dete...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[[[58, 55, 50], [55, 55, 49], [50, 54, 48], [5..."
1737,img_26025.jpg,0,/content/tmp/state-farm-distracted-driver-dete...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[[31, 42, 34], [31, 42, 34], [31, 42, 34], [3..."
9026,img_73781.jpg,3,/content/tmp/state-farm-distracted-driver-dete...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[[[39, 52, 38], [40, 53, 39], [40, 53, 39], [3..."
2120,img_70795.jpg,0,/content/tmp/state-farm-distracted-driver-dete...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[[47, 51, 46], [48, 53, 47], [50, 54, 47], [5..."


In [0]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

x_train = []
x_test = []
y_train = []
y_test= []

for item in train_df['X']:
  x_train.append(item)

for item in validate_df['X']:
  x_test.append(item)

for item in train_df['Y']:
  y_train.append(item)

for item in validate_df['Y']:
  y_test.append(item)

x_train = np.array(x_train, dtype=np.uint8)
x_test = np.array(x_test, dtype=np.uint8)
y_train = np.array(y_train, dtype=np.uint8)
y_test = np.array(y_test, dtype=np.uint8)

In [0]:
#build model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.optimizers import RMSprop, SGD
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

model = Sequential()
model.add(base_model)

model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) # 

for layer in base_model.layers:
    layer.trainable = False

#model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.0001), metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

## Train
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input

earlystop = EarlyStopping(patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
EPOCHS=20
BATCH_SIZE=15
callbacks = [learning_rate_reduction]

history = model.fit(x_train, 
                    y_train,
                    shuffle=True,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test)
                    ,callbacks=callbacks)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
TEST_DIRECTORY = '/content/tmp/state-farm-distracted-driver-detection/imgs/test/batch0'
test_data = pd.DataFrame({'filename': os.listdir(TEST_DIRECTORY)})

test_data_frame = test_data.sample(20, axis=None)
test_data_frame['X'] = test_data_frame['filename'].map(lambda file: get_im_cv2(TEST_DIRECTORY + "/" + file, IMAGE_WIDTH, IMAGE_HEIGHT))

x_perf=[]
for item in test_data_frame['X']:
  x_perf.append(item)

x_perf = np.array(x_perf, dtype=np.uint8)
predict = model.predict(x_perf)



In [0]:
#Plot prediction
import matplotlib.pyplot as plt
label_map = {
    0: "normal driving",
    1: "texting - right",
    2: "talking on the phone - right",
    3: "texting - left",
    4: "talking on the phone - left",
    5: "operating the radio",
    6: "drinking",
    7: "reaching behind",
    8: "hair and makeup",
    9: "talking to passenger"
}

test_data_frame['category'] = np.argmax(predict, axis=1)
test_data_frame['action'] = test_data_frame['category'].map(lambda x: label_map.get(x))
print(test_data_frame.head())

#Draw predictions with images
from keras.preprocessing.image import load_img
sample_test = test_data_frame
plt.figure(figsize=(12, 24))
index=0
for _, row in sample_test.iterrows():
    filename = row['filename']
    category = row['action']
    index=index+1
    img = load_img(TEST_DIRECTORY+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()